In [1]:
!pip install confluent-kafka

In [5]:
pip install schedule

In [2]:
pip install langdetect

In [ ]:
!pip install transformers

In [ ]:
pip install textblob


In [3]:
from confluent_kafka import Producer
import schedule
import time
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta
from langdetect import detect

# Kafka configuration
kafka_config = {
    'bootstrap.servers': 'pkc-lzvrd.us-west4.gcp.confluent.cloud:9092',
    'group.id': 'my-group',
    'auto.offset.reset': 'earliest',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': '7EEPKRB4LLGJOBSC',
    'sasl.password': '0/+d2Vi9vcZOh4g/LJJm4MHs+L6zOt+00LPhiMcTTyjdML5IGP9Wg5Q7aUeWXv2Y'
}

producer = Producer(kafka_config)

# Function to collect and publish data
def collect_and_publish_data():
    root = "https://www.google.com/"

    # Initial link for the first page
    link_base = "https://www.google.com/search?q=tesla&rlz=1C1GCEA_enMA1047MA1047&tbas=0&tbs=qdr:d,sbd:1&tbm=nws&source=lnt&sa=X&ved=2ahUKEwiAmILcsM2AAxWyd6QEHbwdARUQpwV6BAgCEBM&biw=1366&bih=651&dpr=1"
    articles_per_page = 10
    num_pages = 25

    data = []

    # Loop through multiple pages
    for page in range(num_pages):
        start_index = page * articles_per_page
        link = f"{link_base}&start={start_index}"

        req = Request(link, headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'})
        webpage = urlopen(req).read()

        with requests.Session() as c:
            soup = BeautifulSoup(webpage, 'html.parser')
            articles = soup.find_all('a', class_='WlydOe')

        for article in articles:
            time_element = article.find('div', class_='OSrXXb')
            title_element = article.find('div', class_='n0jPhd')
            description_element = article.find('div', class_='GI74Re')

            if time_element:
                time_string = time_element.span.get_text()
                if "min" in time_string:
                    minutes_ago = int(time_string.split()[0])
                    time = datetime.now() - timedelta(minutes=minutes_ago)
                elif "hour" in time_string:
                    hours_ago = int(time_string.split()[0])
                    time = datetime now() - timedelta(hours=hours_ago)
                else:
                    time = "N/A"
            else:
                time = "N/A"

            # Convert time to date and time strings
            date_str = time.strftime("%Y-%m-%d")
            time_str = time.strftime("%H:%M:%S")

            if title_element:
                title = title_element.get_text()
            else:
                title = "N/A"

            if description_element:
                description = description_element.get_text()
            else:
                description = "N/A"

            description_language = detect(description)
            if description_language == 'en':
                data.append({'Date': date_str, 'Time': time_str, 'Title': title, 'Description': description})

    df = pd.DataFrame(data)


    # Convert the data to a string or a specific format (e.g., JSON)
    data_string = df.to_json()  # Convert the DataFrame to a JSON string

    # Publish the data to a Kafka topic
    producer.produce('Tesla1', key='7EEPKRB4LLGJOBSC', value=data_string.encode('utf-8'), value_serializer='raw')

    # Wait for any outstanding messages to be delivered and delivery reports received
    producer.flush()

# Schedule data collection and publication every 30 minutes
schedule.every(30).minutes.do(collect_and_publish_data)

while True:
    schedule.run_pending()
    time.sleep(1)


Data has been scraped and saved to tesla_articles13.xlsx.


0

In [4]:
from confluent_kafka import Consumer, KafkaError
import json
import torch
import pandas as pd
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification
)

# Configuration du consommateur Kafka
kafka_config = {
    'bootstrap.servers': 'pkc-lzvrd.us-west4.gcp.confluent.cloud:9092',
    'group.id': 'my-group',
    'auto.offset.reset': 'earliest',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': '7EEPKRB4LLGJOBSC',
    'sasl.password': '0/+d2Vi9vcZOh4g/LJJm4MHs+L6zOt+00LPhiMcTTyjdML5IGP9Wg5Q7aUeWXv2Y'
}
consumer = Consumer(kafka_config)

# S'abonner au sujet Kafka que vous souhaitez consommer
consumer.subscribe(['VotreSujetKafka'])

# Créer un DataFrame pour stocker les résultats
result_df = pd.DataFrame(columns=['Date', 'Description', 'Sentiment Label', 'Sentiment Score'])

# Fonction pour analyser les sentiments
def analyze_sentiments(text):
    # Set device to GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_id = "cardiffnlp/twitter-roberta-base-sentiment"

    # Load the model from huggingface
    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        num_labels=3
    )

    # Load the tokenizer from huggingface
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load the tokenizer and model into a sentiment analysis pipeline
    nlp = pipeline(
        "sentiment-analysis",
        model=model,
        tokenizer=tokenizer,
        device=device
    )

    labels = {
        "LABEL_0": "negative",
        "LABEL_1": "neutral",
        "LABEL_2": "positive"
    }

    def get_sentiment(reviews):
        reviews = str(reviews)
        sentiments = nlp(reviews)
        sentiment_label = labels[sentiments[0]["label"]]
        sentiment_score = sentiments[0]["score"]

        # Map sentiment label to score between -1 and 1
        if sentiment_label == "negative":
            sentiment_score = -sentiment_score  # Invert score for negative sentiment
        elif sentiment_label == "neutral":
            sentiment_score = 0.01 if sentiment_score > 0.6 else -0.01  # Slightly positive/negative for neutral sentiment
        else:
            sentiment_score = sentiment_score  # Positive sentiment score remains as-is

        return sentiment_label, sentiment_score

    return get_sentiment(text)

try:
    while True:
        # Attendre les messages
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print('Fin de la partition atteinte, continuer...')
            else:
                print('Erreur du consommateur: {}'.format(msg.error()))
        else:
            # Désérialiser le message JSON
            data = json.loads(msg.value())

            # Analyser le sentiment de la description de l'article
            sentiment_label, sentiment_score = analyze_sentiments(data['Description'])

            # Ajouter les résultats au DataFrame
            result_df = result_df.append({'Date': data['Date'], 'Description': data['Description'],
                                          'Sentiment Label': sentiment_label, 'Sentiment Score': sentiment_score},
                                         ignore_index=True)

except KeyboardInterrupt:
    pass

finally:
    # Fermer le consommateur Kafka proprement
    consumer.close()

# Afficher le DataFrame comme une table
print(result_df)

Received message key: b'7EEPKRB4LLGJOBSC', value: b'{"Date":{"0":"2023-08-23","1":"2023-08-23","2":"2023-08-23","3":"2023-08-23","4":"2023-08-23","5":"2023-08-23","6":"2023-08-23","7":"2023-08-23","8":"2023-08-23","9":"2023-08-23","10":"2023-08-23","11":"2023-08-23","12":"2023-08-23","13":"2023-08-23","14":"2023-08-23","15":"2023-08-23","16":"2023-08-23","17":"2023-08-23","18":"2023-08-23","19":"2023-08-23","20":"2023-08-23","21":"2023-08-23","22":"2023-08-23","23":"2023-08-23","24":"2023-08-23","25":"2023-08-23","26":"2023-08-23","27":"2023-08-23","28":"2023-08-23","29":"2023-08-23","30":"2023-08-23","31":"2023-08-23","32":"2023-08-23","33":"2023-08-23","34":"2023-08-23","35":"2023-08-23","36":"2023-08-23","37":"2023-08-23","38":"2023-08-23","39":"2023-08-23","40":"2023-08-23","41":"2023-08-23","42":"2023-08-23","43":"2023-08-23","44":"2023-08-23","45":"2023-08-23","46":"2023-08-23","47":"2023-08-23","48":"2023-08-23","49":"2023-08-23","50":"2023-08-23","51":"2023-08-23","52":"2023-08

Real Time Sentiemnt Analysis
